<a href="https://colab.research.google.com/github/Anurag131277/langchain/blob/main/Evaluation_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install --quiet -U langgraph langchain_community langchain_core langchain_nvidia_ai_endpoints

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.6/133.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.7/415.7 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


In [3]:
import os
from google.colab import userdata

In [4]:
os.environ["NVIDIA_API_KEY"] = userdata.get("NVIDIA_API_KEY")
llm_str= "meta/llama-3.3-70b-instruct"

In [5]:
from langchain.chains import RetrievalQA
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

In [7]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
data = loader.load()

In [8]:
%pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.3 MB/s eta 0:00:00


In [9]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
embeddings = NVIDIAEmbeddings(model="NV-Embed-QA")

In [10]:
%pip install docarray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.2/270.2 kB 14.2 MB/s eta 0:00:00


In [11]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings
).from_loaders([loader])

/usr/local/lib/python3.11/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [12]:
llm = ChatNVIDIA(model=llm_str, temperature=0.0)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=index.vectorstore.as_retriever(),
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

In [ ]:
data[10]

In [ ]:
data[11]

In [30]:
examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

In [13]:
from langchain.evaluation.qa import QAGenerateChain


In [35]:
#QA chain
for t in data[:5]:
  print(t)

page_content=': 0
name: Women's Campside Oxfords
description: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. 

Size & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. 

Specs: Approx. weight: 1 lb.1 oz. per pair. 

Construction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. 

Questions? Please contact us for any inquiries.' metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0}
page_content=': 1
name: Recycled Waterhog Dog Mat, Chevron Weave
description: Protect your floors from spills and splashing with our ultradurable r

In [14]:
example_gen_chain = QAGenerateChain.from_llm(ChatNVIDIA(model=llm_str))

In [32]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)
new_examples

/usr/local/lib/python3.11/dist-packages/langchain/chains/llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[{'qa_pairs': {'query': "What type of material is used for the innersole of the Women's Campside Oxfords, and what feature does it include for odor control?",
   'answer': "The Women's Campside Oxfords feature an EVA innersole with Cleansport NXT antimicrobial odor control."}},
 {'qa_pairs': {'query': 'What is the size of the Medium Recycled Waterhog Dog Mat, and what type of material is used for its backing?',
   'answer': 'The Medium Recycled Waterhog Dog Mat has dimensions of 22.5" x 34.5", and it has a rubber backing.'}},
 {'qa_pairs': {'query': "What percentage of the sun's harmful rays are blocked by the UPF 50+ rated fabric used in the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece?",
   'answer': '98%'}},
 {'qa_pairs': {'query': 'What percentage of Lycra spandex is used in the lining of the Refresh Swimwear V-Neck Tankini Contrasts, and what is the UPF rating of the fabric used in this product?',
   'answer': 'The lining of the Refresh Swimwear V-Neck Tankini Contr

In [38]:
new_examples[0]

{'qa_pairs': {'query': "What type of material is used for the innersole of the Women's Campside Oxfords, and what feature does it include for odor control?",
  'answer': "The Women's Campside Oxfords feature an EVA innersole with Cleansport NXT antimicrobial odor control."}}

In [41]:
test =[{'query': item['qa_pairs']['query'], 'answer': item['qa_pairs']['answer']} for item in new_examples]
test

[{'query': "What type of material is used for the innersole of the Women's Campside Oxfords, and what feature does it include for odor control?",
  'answer': "The Women's Campside Oxfords feature an EVA innersole with Cleansport NXT antimicrobial odor control."},
 {'query': 'What is the size of the Medium Recycled Waterhog Dog Mat, and what type of material is used for its backing?',
  'answer': 'The Medium Recycled Waterhog Dog Mat has dimensions of 22.5" x 34.5", and it has a rubber backing.'},
 {'query': "What percentage of the sun's harmful rays are blocked by the UPF 50+ rated fabric used in the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece?",
  'answer': '98%'},
 {'query': 'What percentage of Lycra spandex is used in the lining of the Refresh Swimwear V-Neck Tankini Contrasts, and what is the UPF rating of the fabric used in this product?',
  'answer': 'The lining of the Refresh Swimwear V-Neck Tankini Contrasts is made with 10% Lycra spandex, and the fabric used in

In [17]:
data[0]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

In [42]:
examples +=test

In [43]:
qa.run(examples[0]["query"])



> Entering new RetrievalQA chain...

> Finished chain.


'Yes, the Cozy Comfort Pullover Set has side pockets, specifically on the pull-on pants.'

In [44]:
#manual evaluation of a llm chain
import langchain
langchain.debug = True

In [45]:
qa.run(examples[0]["query"])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set        have side pockets?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "context": ": 73\nname: Cozy Cuddles Knit Pullover Set\ndescription: Perfect for lounging, this knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out. \n\nSize & Fit \nPants are Favorite Fit: Sits lower on the waist. \nRelaxed Fit: Our most generous fit sits farthest from the body. \n\nFabric & Care \nIn the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features \nRelaxed fit top with raglan sleeves and rounded hem. \nPull-on pants have 

'Yes, the Cozy Comfort Pullover Set has side pockets on the pants. According to the description, the pull-on pants have "side pockets and a modern slim leg".'

In [47]:
langchain.debug= False

In [26]:
examples

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
  'answer': 'Yes'},
 {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection'},
 {'qa_pairs': {'query': "What type of material is used for the innersole of the Women's Campside Oxfords, and what feature does it include for odor control?",
   'answer': "The Women's Campside Oxfords feature an EVA innersole that includes Cleansport NXT® antimicrobial odor control."}},
 {'qa_pairs': {'query': 'What is the material composition of the 24 oz. polyester fabric used in the construction of the Recycled Waterhog Dog Mat, and what is the primary benefit of this composition in terms of environmental impact?',
   'answer': 'The 24 oz. polyester fabric is made from 94% recycled materials, which helps keep plastic out of landfills, trails, and oceans, thereby reducing waste and promoting sustainability.'}},
 {'qa_pairs': {'query': "What percentage of the su

In [58]:
predictions = qa.apply(examples)
predictions



> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
  'answer': 'Yes',
  'result': 'Yes, the Cozy Comfort Pullover Set has side pockets, specifically on the pull-on pants.'},
 {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection',
  'result': 'The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.'},
 {'query': "What type of material is used for the innersole of the Women's Campside Oxfords, and what feature does it include for odor control?",
  'answer': "The Women's Campside Oxfords feature an EVA innersole with Cleansport NXT antimicrobial odor control.",
  'result': "The Women's Campside Oxfords have a comfortable EVA (Ethylene-Vinyl Acetate) innersole, which includes Cleansport NXT antimicrobial odor control to help prevent foot odor."},
 {'query': 'What is the size of the Medium Recycled Waterhog Dog Mat, and what type of material is used for its backing?',
  '

In [49]:
from langchain.evaluation.qa import QAEvalChain

In [50]:
llm = ChatNVIDIA(temperature=0, model=llm_str)
eval_chain = QAEvalChain.from_llm(llm)

In [59]:
graded_outputs = eval_chain.evaluate(examples, predictions)

In [60]:
graded_outputs
# predictions

[{'results': 'GRADE: CORRECT'},
 {'results': 'GRADE: CORRECT'},
 {'results': 'GRADE: CORRECT'},
 {'results': 'GRADE: CORRECT'},
 {'results': 'GRADE: CORRECT'},
 {'results': 'GRADE: CORRECT \n\nThe student answer contains the correct percentage of Lycra spandex (10%) and the correct UPF rating (50+), which matches the true answer. The additional information about the lining being made from 90% recycled nylon does not conflict with the true answer, so it does not affect the grade.'},
 {'results': 'GRADE: CORRECT'}]

In [61]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['results'])
    print()

Example 0:
Question: Do the Cozy Comfort Pullover Set        have side pockets?
Real Answer: Yes
Predicted Answer: Yes, the Cozy Comfort Pullover Set has side pockets, specifically on the pull-on pants.
Predicted Grade: GRADE: CORRECT

Example 1:
Question: What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?
Real Answer: The DownTek collection
Predicted Answer: The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.
Predicted Grade: GRADE: CORRECT

Example 2:
Question: What type of material is used for the innersole of the Women's Campside Oxfords, and what feature does it include for odor control?
Real Answer: The Women's Campside Oxfords feature an EVA innersole with Cleansport NXT antimicrobial odor control.
Predicted Answer: The Women's Campside Oxfords have a comfortable EVA (Ethylene-Vinyl Acetate) innersole, which includes Cleansport NXT antimicrobial odor control to help prevent foot odor.
Predicted Grade: GRADE: CORRECT

Exampl